In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Q1.  Who are the top 5 users in Chicago
 with the highest number of

1.   List item
2.   List item

followers? List their login in order, comma-separated

In [ ]:
users = pd.read_csv('users.csv')
users.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,brianyu28,Brian Yu,NaN,"Boston, MA",brian@brianyu.me,False,Software developer and educator,35,13198,13,2015-11-29T07:25:29Z
1,PatrickAlphaC,Patrick Collins,CYFRIN,Boston,NaN,False,"Smart Contract Engineer, Auditor, and Educator",272,9661,43,2019-08-19T14:13:41Z
2,KeithGalli,Keith Galli,NaN,"Boston, MA",NaN,True,YouTube Content Creator :).,52,5677,1,2013-12-25T19:49:26Z
3,CharlesCreativeContent,Shawn Charles,AMAZON,"Boston, MA",NaN,True,Software Engineer building Tech Communities,83,5056,1092,2020-03-11T20:10:11Z
4,timbl,Tim Berners-Lee,INRUPT,Boston MA USA,timbl@w3.org,False,NaN,18,4849,69,2011-12-11T01:28:03Z


In [ ]:
users['hireable'] = users['hireable'].fillna(False).astype(bool)

In [ ]:
top5 = users.sort_values(by='followers', ascending=False).head()
print(','.join(top5['login'].tolist()))

brianyu28,PatrickAlphaC,KeithGalli,CharlesCreativeContent,timbl


#### Q2. Who are the 5 earliest registered GitHub users in Chicago? List their login in ascending order of created_at, comma-separated.

In [ ]:
users['created_at'] = pd.to_datetime(users['created_at'])

In [ ]:
top_earliest = users.sort_values(by='created_at').head()
print(','.join(top_earliest['login'].tolist()))

evan,dpickett,tel,radical,joshuaclayton


#### Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [ ]:
repos = pd.read_csv('repositories.csv')
repos.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,brianyu28,brianyu28/cs50,2019-09-17T15:00:36Z,19,19,HTML,True,True,NaN
1,brianyu28,brianyu28/scratch-to-blocks,2020-05-02T11:08:11Z,11,11,Python,True,True,NaN
2,brianyu28,brianyu28/holyoke,2022-06-09T00:21:08Z,6,6,Swift,True,True,NaN
3,brianyu28,brianyu28/dispatch,2017-08-26T21:43:51Z,39,39,Rust,True,True,gpl-3.0
4,brianyu28,brianyu28/multicolor,2024-04-08T22:45:21Z,5,5,TypeScript,True,True,NaN


In [ ]:
repos['license_name'].value_counts().head(3)

license_name
mit           9898
other         4661
apache-2.0    3559
Name: count, dtype: int64

#### Q4. Which company do the majority of these developers work at?

In [ ]:
users['company'].value_counts().head(1)

company
NORTHEASTERN UNIVERSITY    16
Name: count, dtype: int64

#### Q5. Which programming language is most popular among these users?

In [ ]:
repos['language'].value_counts().head(1)

language
JavaScript    7695
Name: count, dtype: int64

#### Q6. Which programming language is the second most popular among users who joined after 2020?

In [ ]:
users_after_2020 = users[users['created_at'] > '2020-01-01']
users_after_2020.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
3,CharlesCreativeContent,Shawn Charles,AMAZON,"Boston, MA",NaN,True,Software Engineer building Tech Communities,83,5056,1092,2020-03-11 20:10:11+00:00
47,neuralmagic,Neural Magic,NaN,Boston,NaN,False,Neural Magic helps developers in accelerating ...,57,762,0,2020-07-23 00:16:47+00:00
101,Lanny-MacMillan,Lanny MacMillan,THRYV,"Boston, Ma",NaN,False,Software Developer / Game Developer\r\nReact.j...,22,396,719,2022-04-01 18:10:46+00:00
109,ibrahimgurhandev,Ibrahim Gurhan,RESILIENT CODERS,Boston MA,NaN,False,Software Engineer at Resilient Coders | Web D...,36,361,798,2020-07-20 04:11:30+00:00
123,jessicajaniuk,Jessica Janiuk,GOOGLE,"Boston, MA",NaN,False,This account is retired. Find Jessica at githu...,6,329,0,2020-10-12 21:49:39+00:00


In [ ]:
repos_2020 = repos[repos['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()

language
JavaScript    98
Python        86
TypeScript    38
HTML          34
CSS           14
Name: count, dtype: int64

#### Q7. Which language has the highest average number of stars per repository?

In [ ]:
avg_stars = repos.groupby('language')['stargazers_count'].mean()
top_lang = avg_stars.idxmax()
top_stars = avg_stars.max()
print(top_lang, top_stars)

SQL 714.0


#### Q8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [ ]:
users['leader_strength'] = users['followers'] / (1 + users['following'])
top5_lead = users.sort_values(by='leader_strength', ascending=False).head()
print(','.join(top5_lead['login'].tolist()))

nikomatsakis,ccoenraets,KeithGalli,rstudio,pluskid


#### Q9. What is the correlation between the number of followers and the number of public repositories among users in  Chicago?

In [ ]:
correlation = users['followers'].corr(users['public_repos'])
correlation

0.16798072850061646

#### Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [ ]:
import csv
followers = []
public_repos = []
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        followers_count = int(row['followers'])
        public_repos_count = int(row['public_repos'])
        followers.append(followers_count)
        public_repos.append(public_repos_count)
if len(followers) > 1 and len(public_repos) > 1:
    slope, intercept = np.polyfit(public_repos, followers, 1)

    print(f"{slope:.3f}")
else:
    print("Error")

1.189


#### Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [ ]:
if repos['has_projects'].dtype == 'object':
    repos['has_projects'] = repos['has_projects'].map({'true': True, 'false': False})
if repos['has_wiki'].dtype == 'object':
    repos['has_wiki'] = repos['has_wiki'].map({'true': True, 'false': False})

correlation = repos['has_projects'].corr(repos['has_wiki'])

print(round(correlation, 3))

0.324


#### Q12. Do hireable users follow more people than those who are not hireable?

In [ ]:
hireable_avg_following = users[users['hireable'] == True]['following'].mean()
non_hireable_avg_following = users[users['hireable'] == False]['following'].mean()
difference = hireable_avg_following - non_hireable_avg_following
difference

111.3778537963858

#### Q13. Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode characters) with followers? (Ignore people without bios)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
# Load the CSV file into a DataFrame
users = pd.read_csv('users.csv')
# Filter users with non-empty 'bio' and create a new column 'bio_len' for the length of the bio
users_with_bio = users[(users['bio'].notna()) & (users['bio'] != '')].copy()
users_with_bio.loc[:, 'bio_len'] = users_with_bio['bio'].str.len()
# Prepare the data for linear regression
X = users_with_bio['bio_len'].values.reshape(-1, 1)
y = users_with_bio['followers']
# Perform linear regression
lr2 = LinearRegression()
lr2.fit(X, y)

# Print the coefficient
print(lr2.coef_[0])


-0.8112187484833792


#### Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [ ]:
import csv
from collections import Counter
from datetime import datetime

weekend_repo_counts = Counter()

with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        created_at = row.get('created_at', '')
        if created_at:
            created_date = datetime.fromisoformat(created_at[:-1])

            if created_date.weekday() in [5, 6]:
                user_login = row['login']
                weekend_repo_counts[user_login] += 1

top_users = weekend_repo_counts.most_common(5)

top_logins = [user[0] for user in top_users]

print(','.join(top_logins))

cameronraysmith,berquist,burtbeckwith,jimkang,johnny-rice


#### Q15. Do people who are hireable share their email addresses more often?

In [ ]:
fraction_hierable = users[users['hireable'] == True]['email'].notna().mean()
fraction_non_hierable = users[users['hireable'] == False]['email'].notna().mean()
diff = fraction_hierable - fraction_non_hierable
diff

0.11672987779182475

#### Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [ ]:
new_users = users[users['name'].notna()].copy()
new_users['surname'] = new_users['name'].str.split().str[-1].str.strip()
surname_counts = new_users['surname'].value_counts()
max_count = surname_counts.max()
common_surnames = surname_counts[surname_counts == max_count].index.tolist()
common_surnames.sort()
print(','.join(common_surnames))

Williams


In [ ]:
# users = users.fillna("").astype(str)
# repos = repos.fillna("").astype(str)

# repos['has_wiki'] = repos['has_wiki'].astype(str).replace({'True': 'true', 'False': 'false'})
# repos['has_projects'] = repos['has_projects'].astype(str).replace({'True': 'true', 'False': 'false'})

# # users.to_csv('users1.csv', index=False)
# repos.to_csv('repos1.csv', index=False)